In [2]:
import sys
import os
# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

In [3]:
import pandas as pd
from textblob   import TextBlob
import sys
import os
from scripts.correlation import load_stock_data, custom_aggregation, get_sentiment_score

In [4]:
stock_price, news_article = load_stock_data("../Data", "../Data/news_article.csv")

In [5]:
merged_df = custom_aggregation(news_article, stock_price)

In [6]:
merged_df.shape

(18178, 13)

In [7]:
merged_df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,headline,publisher,url,stock
18173,2020-06-05,58.522667,59.101334,57.746666,59.043999,59.043999,117178500,0.0,0.0,Stocks That Hit 52-Week Highs On Friday | 10 B...,"Benzinga Insights, Benzinga Newsdesk, Benzinga...",https://www.benzinga.com/news/20/06/16190091/s...,"A, AA, AAN, AAWW, ABC, ABMD, ABR, ACC, ACGL, A..."
18174,2020-06-08,61.266666,63.333332,60.610668,63.327999,63.327999,212620500,0.0,0.0,Alcoa Seeing Better July Aluminum Shipments Am...,"Benzinga Newsdesk, Benzinga Newsdesk, Luke J J...",https://www.benzinga.com/news/20/06/16203021/a...,"AA, AAL, AAN, AAWW, ABBV, ABEV, ABR, ACOR, ACT..."
18175,2020-06-09,62.667332,63.629333,61.595333,62.711334,62.711334,170823000,0.0,0.0,Shares of several basic material companies are...,"Benzinga Newsdesk, Benzinga Newsdesk, Tanzeel ...",https://www.benzinga.com/wiim/20/06/16213027/s...,"AA, AAL, AAPL, ACGL, ACY, AEE, AER, AES, AEY, ..."
18176,2020-06-10,66.125336,68.498665,65.500000,68.336670,68.336670,278451000,0.0,0.0,International Air Travel On U.S. Airlines Fell...,"Benzinga Newsdesk, Benzinga Newsdesk, JJ Kinah...",https://www.benzinga.com/news/20/06/16223284/i...,"AAL, AAN, AAP, AAPL, ABBV, ABIO, ACGL, ACRX, A..."
18177,2020-06-11,66.013336,67.930664,64.800003,64.856003,64.856003,238747500,0.0,0.0,Johnson & Johnson To Start Coronavirus Vaccine...,"Neer Varshney, Lisa Levin, Charles Gross, Benz...",https://www.benzinga.com/general/biotech/20/06...,"AZN, CANF, CHK, CHS, CNHI, DHI, DPW, EOG, EPR,..."


In [8]:
# Apply sentiment analysis to headlines
merged_df['sentiment_score'] = merged_df['headline'].apply(get_sentiment_score)

In [9]:
# Group by date and calculate the average sentiment score
daily_sentiment = merged_df.groupby('Date')['sentiment_score'].mean().reset_index()

In [10]:
merged_df['daily_return'] = merged_df['Close'].pct_change()

In [11]:
# Merge daily sentiment with daily stock returns
merged_daily_df = pd.merge(daily_sentiment, merged_df[['Date', 'daily_return']], on='Date', how='inner')
merged_daily_df.drop_duplicates(subset='Date', inplace=True)

In [12]:
# Calculate correlation
correlation = merged_daily_df['sentiment_score'].corr(merged_daily_df['daily_return'])
print(f"Correlation between sentiment scores and stock daily returns: {correlation:.4f}")

Correlation between sentiment scores and stock daily returns: 0.1017
